In [ ]:

#import all libraries we used

import pandas as pd
from bs4 import BeautifulSoup
import requests
import html5lib
import os
import re

In [ ]:

#create our DataFrame

def createDf():
    data = {'round': [], 'season': [], 'date':[], 'hour':[], 'home team': [], 'guest team': [], 'score': [],'winners': []}
    df = pd.DataFrame(data)
    return df

In [ ]:

#Create a string of two seasons

def season(firstY,secondY):
    years = firstY+"-"+secondY
    return years

In [ ]:

#Create BeautifulSoup object

def load_soup_object(years,rounds):
    if years == "2016-2017":
        url = "https://www.worldfootball.net/schedule/esp-primera-division-"+years+"-spieltag_2/"+rounds
    else:
        url = "https://www.worldfootball.net/schedule/esp-primera-division-"+years+"-spieltag/"+rounds
    r = requests.get(url)
    soup = BeautifulSoup(r.text,'html.parser')
    return soup

In [ ]:

#Insert our BeautifulSoup object to our Dataframe

def insertSoupToDataFrame(soup,df,rounds,years):
    
    date=list()
    hour=list()
    homeTeam=list()
    guestTeam=list()
    score=list()
    
    table = soup.find('table',attrs={"class":"standard_tabelle"})
    
    for row in table("tr"):
        cells = row("td")
        if(len(cells)<6):
            continue
        if cells[0].get_text() != "":
            temp = cells[0].get_text()
            date.append(cells[0].get_text())
        else:
            date.append(temp)
        hour.append(cells[1].get_text())
        homeTeam.append(cells[2].get_text().strip())
        guestTeam.append(cells[4].get_text().strip())
        score.append(cells[5].get_text().strip())
        
        
    data = {'round': rounds, 'season': years, 'date':date, 'hour':hour, 'home team': homeTeam, 'guest team': guestTeam, 'score': score}
    df = pd.DataFrame(data)
    return df
    

In [ ]:

#Our main function to get our data using the last functions of creatring BeautifulSoup and dataframe

def getData():
    yearList = ["2000", "2001", "2002","2003", "2004", "2005","2006", "2007", "2008","2009",
                "2010", "2011","2012", "2013", "2014","2015", "2016", "2017","2018", "2019", "2020","2021", "2022"]

    roundsList = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10",
                  "11", "12","13", "14", "15", "16", "17", "18", "19", "20",
                  "21", "22", "23", "24", "25", "26", "27", "28", "29", "30",
                  "31", "32", "33", "34", "35", "36", "37", "38"]

    df = createDf()
    tempYear = 0;
    tempRounds = 0;
    for year in yearList:
        if yearList[tempYear] == "2022" :
            break
        years = season(yearList[tempYear],yearList[tempYear+1])
        for rounds in roundsList:
            soup = load_soup_object(years,rounds)
            df1 = insertSoupToDataFrame(soup,df,rounds,years)
            df = df.append(df1, ignore_index=True)
            tempRounds = tempRounds+1
        tempYear=tempYear+1
    
    return df

In [ ]:

#Check the score and fill winners column 

def getWinners(df):
    df1 = df.copy()
    temp = df.score
    winners = list()
    for i in temp:
        if i[0] < i[2]:
            winners.append("Away")
        elif i[0] > i[2]:
            winners.append("Home")
        else:
            winners.append("Draw")
    df1.winners = winners
    return df1

In [ ]:

#Edit the score 

def editScore(df):
    df1 = df.copy()
    temp = df.score
    scoreEdit = list()
    for i in temp:
            i = i[0: 3]
            scoreEdit.append(i)
    df1.score = scoreEdit
    return df1

In [ ]:

#update empty hours with the value 0

def updateHours(df):
    df1 = df.copy()
    temp = df.hour
    hours = list()
    for i in temp:
        if i=="NaN" or i == "":
            hours.append("0")
        else:
            hours.append(i)
    df1.hour = hours
    return df1

In [ ]:
df = getData()
df1 = getWinners(df)
df2 = updateHours(df1)
df3 = editScore(df2)
df3.to_csv('data.csv')
df3

In [ ]:
df = pd.read_csv('data.csv')
df1 = df.drop(['Unnamed: 0'], axis=1)
df2 = df1.dropna(axis=0)
df2